In [1]:
import pandas as pd
import os
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [3]:
baojiadan_folder = 'D:\\Program Files (x86)\\百度云同步盘\\Dropbox\\-E·J- 2014.5.1\\2015.9.26 燕文\\'

In [4]:
baojiadan_file_list = list()

In [5]:
for file_name in os.listdir(baojiadan_folder):
    if file_name.startswith('广州燕文WISH报价单'):
        baojiadan_file_list.append(file_name)

In [6]:
latest_baojiadan_file_name = baojiadan_file_list[-1]
# latest_baojiadan_file_name

In [7]:
latest_baojiadan_path = os.path.join(baojiadan_folder, latest_baojiadan_file_name)
# latest_baojiadan_path

In [8]:
baojiadan_sheets_dict = pd.read_excel(latest_baojiadan_path, sheetname=None)
# type(baojiadan)

C:\Users\Enzo.kuang\Anaconda3\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [9]:
wanted_shipping_service_list = ['WISH燕文航空挂号小包-普货',
                                'WISH燕文航空挂号小包-特货',
                                'WISH燕文专线追踪小包-普货',
                                'WISH燕文专线追踪小包-特货',
                                'WISH燕邮宝挂号-特货',
                                'WISH燕文航空易派小包-普货',
                                'WISH燕文航空易派小包-特货',
                                'WISH燕文C挂号小包',
                                'WISH燕文C优先小包'
]

In [10]:
export_path = os.path.join(baojiadan_folder, '报价单','燕文（wish）报价表格.xlsx')
writer = pd.ExcelWriter(export_path)

for service_name in wanted_shipping_service_list:
    if service_name in baojiadan_sheets_dict.keys():
        logging.info(service_name + ' is ready to export.')
        df = baojiadan_sheets_dict[service_name]
        try:
            for c in df.columns:
                for index, r in enumerate(df[c]):
                    if r == '国家':
                        country_cell_location = [index, c]
                        # print(country_cell_location)
                        break
            df = df[country_cell_location[0]:]
            df = df.dropna(how='all')
            df = df[df[country_cell_location[1]].str.len() < 10]
            
            df.to_excel(writer, sheet_name=service_name, index=False)
            logging.info(service_name + ' exported.')
            # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.csv'))
        except:
            logging.info(service_name + ' failed to export.')
            pass
        # df.to_csv(os.path.join(baojiadan_folder, '报价单', key + '.xlsx'))
        
writer.save()

 2019-02-25 12:39:29,217 - INFO - WISH燕文航空挂号小包-普货 is ready to export.
 2019-02-25 12:39:29,233 - INFO - WISH燕文航空挂号小包-普货 exported.
 2019-02-25 12:39:29,234 - INFO - WISH燕文航空挂号小包-特货 is ready to export.
 2019-02-25 12:39:29,244 - INFO - WISH燕文航空挂号小包-特货 exported.
 2019-02-25 12:39:29,244 - INFO - WISH燕文专线追踪小包-普货 is ready to export.
 2019-02-25 12:39:29,248 - INFO - WISH燕文专线追踪小包-普货 exported.
 2019-02-25 12:39:29,249 - INFO - WISH燕文专线追踪小包-特货 is ready to export.
 2019-02-25 12:39:29,254 - INFO - WISH燕文专线追踪小包-特货 exported.
 2019-02-25 12:39:29,255 - INFO - WISH燕邮宝挂号-特货 is ready to export.
 2019-02-25 12:39:29,258 - INFO - WISH燕邮宝挂号-特货 exported.
 2019-02-25 12:39:29,259 - INFO - WISH燕文航空易派小包-普货 is ready to export.
 2019-02-25 12:39:29,264 - INFO - WISH燕文航空易派小包-普货 exported.
 2019-02-25 12:39:29,264 - INFO - WISH燕文航空易派小包-特货 is ready to export.
 2019-02-25 12:39:29,269 - INFO - WISH燕文航空易派小包-特货 exported.
 2019-02-25 12:39:29,269 - INFO - WISH燕文C挂号小包 is ready to export.
 2019-02-25 12:39:29,297 - INF